In [1]:
from sklearn import datasets
import pandas as pd
import numpy as np

In [20]:
# Load the diabetes dataset from sklearn
df = pd.read_csv("DelayedFlights.csv", index_col=0)

## Drop information that known after the flight start

In [3]:
non_trainable_columns = [
    "DepTime", "ArrTime", "ActualElapsedTime",\
    "DepDelay", "TaxiIn", "TaxiOut", "Cancelled",\
    "CancellationCode", "Diverted", "CarrierDelay", "WeatherDelay",\
    "NASDelay", "SecurityDelay", "LateAircraftDelay"\
]

In [4]:
df = df.drop(non_trainable_columns, axis=1)

## Dealing with Null values

In [5]:
df.isna().sum()

Year                 0
Month                0
DayofMonth           0
DayOfWeek            0
CRSDepTime           0
CRSArrTime           0
UniqueCarrier        0
FlightNum            0
TailNum              5
CRSElapsedTime     198
AirTime           8387
ArrDelay          8387
Origin               0
Dest                 0
Distance             0
dtype: int64

In [6]:
df = df.dropna()

In [7]:
df.head()

,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,CRSElapsedTime,AirTime,ArrDelay,Origin,Dest,Distance
0,2008,1,3,4,1955,2225,WN,335,N712SW,150.0,116.0,-14.0,IAD,TPA,810
1,2008,1,3,4,735,1000,WN,3231,N772SW,145.0,113.0,2.0,IAD,TPA,810
2,2008,1,3,4,620,750,WN,448,N428WN,90.0,76.0,14.0,IND,BWI,515
3,2008,1,3,4,1755,1925,WN,3920,N464WN,90.0,77.0,34.0,IND,BWI,515
4,2008,1,3,4,1915,2110,WN,378,N726SW,115.0,87.0,11.0,IND,JAX,688


## Convert float columns to int

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1928368 entries, 0 to 1936757
Data columns (total 15 columns):
 #   Column          Dtype  
---  ------          -----  
 0   Year            int64  
 1   Month           int64  
 2   DayofMonth      int64  
 3   DayOfWeek       int64  
 4   CRSDepTime      int64  
 5   CRSArrTime      int64  
 6   UniqueCarrier   object 
 7   FlightNum       int64  
 8   TailNum         object 
 9   CRSElapsedTime  float64
 10  AirTime         float64
 11  ArrDelay        float64
 12  Origin          object 
 13  Dest            object 
 14  Distance        int64  
dtypes: float64(3), int64(8), object(4)
memory usage: 235.4+ MB


In [9]:
df[['CRSElapsedTime', 'AirTime', 'ArrDelay']] = df[['CRSElapsedTime', 'AirTime', 'ArrDelay']].astype(int)

## Convert time from `hhmm` to `total minutes` format

In [10]:
time_features = ["CRSDepTime", "CRSArrTime"]

In [11]:
def hhmm_to_total_minutes(hhmm: int) -> int:
    return ((hhmm // 100) * 60) + (hhmm % 100)

In [12]:
df[time_features] = df[time_features].apply(hhmm_to_total_minutes)

## Encode categorical features

In [13]:
df.head()

,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,CRSElapsedTime,AirTime,ArrDelay,Origin,Dest,Distance
0,2008,1,3,4,1195,1345,WN,335,N712SW,150,116,-14,IAD,TPA,810
1,2008,1,3,4,455,600,WN,3231,N772SW,145,113,2,IAD,TPA,810
2,2008,1,3,4,380,470,WN,448,N428WN,90,76,14,IND,BWI,515
3,2008,1,3,4,1075,1165,WN,3920,N464WN,90,77,34,IND,BWI,515
4,2008,1,3,4,1155,1270,WN,378,N726SW,115,87,11,IND,JAX,688


In [14]:
categorical_features = ["UniqueCarrier", "TailNum", "Origin", "Dest"]
df[categorical_features].nunique()

UniqueCarrier      20
TailNum          5360
Origin            303
Dest              302
dtype: int64

In [15]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for feature in categorical_features:
    df[f"{feature}_encoded"] = le.fit_transform(df[feature])

## Convert ArrDelay column into boolean indicates wheather flight arrival late or not

In [16]:
df["IsArrLate"] = (df["ArrDelay"] > 0)

## Save processed data

In [17]:
df = df.drop(categorical_features, axis=1)
df = df.drop("ArrDelay", axis=1)

df.to_csv("CleanedDelayedFlights.csv")

In [19]:
df.head()

,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,FlightNum,CRSElapsedTime,AirTime,Distance,UniqueCarrier_encoded,TailNum_encoded,Origin_encoded,Dest_encoded,IsArrLate
0,2008,1,3,4,1195,1345,335,150,116,810,17,3756,135,284,False
1,2008,1,3,4,455,600,3231,145,113,810,17,4116,135,284,True
2,2008,1,3,4,380,470,448,90,76,515,17,1948,140,48,True
3,2008,1,3,4,1075,1165,3920,90,77,515,17,2129,140,48,True
4,2008,1,3,4,1155,1270,378,115,87,688,17,3839,140,149,True
